In [1]:
##### 16p chord diagram del vs con 20% thresholded , binary color / July 2018

In [16]:
library("xlsx")
library("circlize")
library('gdata')
library('Matrix')
library('RColorBrewer')
library('viridis')
library('R.matlab')

In [17]:
setwd("/project/6008022/su_cm/paper_16p22q/glm/16p/cambridge64/")
mat_p = 'glm_results.mat' ## to generate before with the matlab script 'Glm_to_CirclePlot.m'
label_p = '/project/6008022/su_cm/paper_16p22q/parcellation/summary_label_64.xlsx'
threshold = 0.2
thr_fdr = TRUE

In [18]:
# get data
mat <- readMat(mat_p)
delcon_eff <- mat$delcon.eff
delcon_fdr <- mat$delcon.fdr
label <- read.xlsx(label_p, 1, header = T)

In [19]:
# Relabel some networks
label$good_networks <- NA
label$good_networks[label$s12_label=='BG_THAL'] <- 'Basal Ganglia'
label$good_networks[label$s12_label=='AUDnet_PINS'] <- 'Auditory'
label$good_networks[label$s12_label=='LIMnet'] <- 'Limbic'
label$good_networks[label$s12_label=='CER'] <- 'Cerebellum'
label$good_networks[label$s12_label=='VATTnet_SALnet'] <- 'Ventral Attention'
label$good_networks[label$s12_label=='FPnet'] <- 'Fronto Parietal'
label$good_networks[label$s12_label=='MOTnet'] <- 'Motor'
label$good_networks[label$s12_label=='VVIS_DVIS'] <- 'Downstream Visual'
label$good_networks[label$s12_label=='DMnet_pm'] <- 'p. DMN'
label$good_networks[label$s12_label=='DMnet_am_lhAG'] <- 'medial DMN'
label$good_networks[label$s12_label=='DMnet_l'] <- 'lateral DMN'
label$good_networks[label$s12_label=='VISnet'] <- 'Visual'

ordered <- c('Auditory', 'Basal Ganglia', 'Cerebellum', 'medial DMN', 'lateral DMN',
             'p. DMN', 'Fronto Parietal', 'Limbic', 'Motor', 'Ventral Attention', 
             'Visual', 'Downstream Visual')

label$good_networks <- factor(label$good_networks, levels=ordered)

In [20]:
#make dataframe
dc_e <- as.matrix(delcon_eff)
rownames(dc_e) <- label$roi_label
colnames(dc_e) <- label$roi_label

dc_f <- as.matrix(delcon_fdr)
rownames(dc_f) <- label$roi_label
colnames(dc_f) <- label$roi_label

# Make simple ord_names
ord_names <- label[label$s12_ID==1,]$roi_label
for (i in 2:12) {
  ord_names <- append(as.character(ord_names), as.character(label[label$s12_ID==i,]$roi_label))
}

# Mask the matrix
if (thr_fdr) {
  dc_e[dc_f>0.05] <- 0
}

dc_conn = upperTriangle(dc_e, diag=FALSE)
dc_nz = abs(dc_conn[dc_conn!=0])
dc_ind = sort(dc_nz, index.return=TRUE, decreasing = TRUE)
dc_slice = floor(length(dc_ind$ix)*threshold)
dc_thr = min(dc_nz[dc_ind$ix[1:dc_slice]])

# Threshold the connections
dc_e[abs(dc_e)<dc_thr] <- 0

# Find regions with no above threshold connections
dc_i <- which(rowSums(dc_e)==0)

# Give them some magic by finding the smallest connection sum and divide by the number of connections
dc_m <-min(rowSums(dc_e)[rowSums(dc_e)>0])/64

# Now give this magic to the nodes without above threshold connections
dc_e[dc_i,] <- dc_m
dc_e[, dc_i] <- dc_m

tn <- colnames(dc_e)
col_neg <- "#5e96d1aa"
col_pos <- "#c64970aa"

In [21]:
colors <- c('#ffffb3','#8dd3c7','#ffed6f','#fccde5','#fb8072','#fdb462','#bc80bd','#bebada','#b3de69','#ccebc5','#80b1d3','#d9d9d9')

In [22]:
# Find regions connected to target
target <- 'CAUDN'
# Get the column index of the target
target_ind <- match(target, colnames(dc_e))
conn <- colnames(dc_e)[(as.vector(dc_e[colnames(dc_e)==target,]))>dc_m]
# Get the number of connections
n_conn <- length(conn)
# Get their index
conn_ind <- match(conn, colnames(dc_e))

# Preallocate connection vectors
to <- c(rep(NA, n_conn))
from <- c(rep(NA, n_conn))

to_tmp <- colnames(dc_e)[(as.vector(dc_e[colnames(dc_e)=='CAUDN',]))>dc_m]
from <- c(rep('CAUDN', length(to)))

In [23]:
n_conn
for (i in 1:n_conn) {
    # See if the connection is row or column
    if (conn_ind[i] > target_ind) {
        to[i] <- target
        from[i] <- conn[i]
    }
    else {
        to[i] <- conn[i]
        from[i] <- target
    }
}

[1] 14

In [24]:
dc_e_t <- dc_e
dc_e_t[dc_e_t <= dc_m] <- 0
matrix<- dc_e_t[order(dc_e_t[,"CAUDN"]),]
#matrix[,4]
n_conn
col_df = data.frame(from, to, col_pos)
col_df

[1] 14

from,to,col_pos
CAUDN,THAL,#c64970aa
CAUDN,MOTnet_vl,#c64970aa
MTgyr_a,CAUDN,#c64970aa
OCCTgyr_l,CAUDN,#c64970aa
IMsul,CAUDN,#c64970aa
PGACcor,CAUDN,#c64970aa
AINS_v_PIsul,CAUDN,#c64970aa
MOTnet_m,CAUDN,#c64970aa
AUDnet,CAUDN,#c64970aa
DMPFcor_a,CAUDN,#c64970aa


In [25]:
setwd("/project/6008022/su_cm/paper_16p22q/figures/fig2/")

pdf("circleplot_16pDEL_CAUDN.pdf",width=16,height=16, bg="white") 

mar.default <- c(5,5,5,5) + 0.1
#par(mar = mar.default + c(0, 4, 0, 0)) 
par(mar = mar.default)

chordDiagram(dc_e, col=col_df, grid.col = "grey25", order = ord_names, keep.diagonal = FALSE, symmetric=TRUE, preAllocateTracks = list(track.height = 0.05), annotationTrack = "grid",transparency = 0.5 )
net_names <- levels(label$good_networks)
for (i in seq_along(net_names)) {
  name <- net_names[i]
  rois <- label[label$good_networks==name,]$ROI
  highlight.sector(tn[rois], track.index = 1, col = colors[i], cex = 0.9, niceFacing = TRUE)
  #highlight.sector(tn[rois], track.index = 1, col = colors[i],
  #                 text = name, cex = 0.9, text.col = "black", niceFacing = TRUE)
}
circos.track(track.index = 1, panel.fun = function(x, y) {
  xlim = get.cell.meta.data("xlim")
  xplot = get.cell.meta.data("xplot")
  ylim = get.cell.meta.data("ylim")
  sector.name = get.cell.meta.data("sector.index")
  #circos.text(mean(xlim), ylim[2]+1, sector.name, facing = "clockwise",
  #            niceFacing = TRUE, cex = 0.85, adj = c(0.1, 0.5), col = "black") 
  #niceFacing = TRUE, cex = 0.7, adj = c(0, 0.5), col = "black") 
  
}, bg.border = NA)
dev.off()

png 
  2